# Compute acoustic indices

## 1. Load packages

In [1]:
# -*- coding: utf-8 -*-
"""
Updated 20 October 2025
Author: Sylvain Haupert
"""

from IPython import get_ipython
print(__doc__)

# Clear all the variables
get_ipython().run_line_magic('reset', '-sf')

# suppress all warnings
import warnings
warnings.filterwarnings("ignore")

# Import standard libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
import sys
import time

# Parallel processing packages
from tqdm import tqdm
from concurrent import futures

# maad package to process the sound and extract the acoustic indices
from maad import sound, features, util

# Import configuration file
import sys
sys.path.append(str(Path('../src')))
import config as cfg



Updated 20 October 2025
Author: Sylvain Haupert



## 2. Notebook init and options setting

* Select the option and the configuration file
* init the local function

In [ ]:
"""==============================================================================
    options             
=============================================================================="""

PROCESS_DATA = True
SAVE = True
DISPLAY = True
VERBOSE = False
CONFIG = cfg.load_config('config_publication.yaml')

In [ ]:
"""==============================================================================
    local function to compute acoustic indices 
=============================================================================="""

def single_file_processing(audio_full_filename, date_time):
    """
    Compute acoustic indices for a single audio file.

    Summary of the steps:
    ---------------------
    - load the sound file
    - resample the sound to the target sampling frequency
    - bandpass filter the sound between flim_min and flim_max
    - trim the sound between tlim_min and tlim_max
    - compute temporal indices
    - compute spectral indices
    - create a dataframe with all the indices

    Parameters
    ----------
    audio_full_filename : str
        Full path to the audio file.
    date_time : str
        Date and time associated with the audio file.

    Returns
    -------
    df_indices : pandas.DataFrame
        DataFrame containing the computed acoustic indices.
    
    """

    _, filename_with_ext = os.path.split(audio_full_filename)
    file = os.path.splitext(filename_with_ext)[0]

    # Load the original sound (16bits) and get the sampling frequency fs
    try:
        wave, fs = sound.load(
            filename=audio_full_filename,
            channel=CONFIG['channel'], 
            detrend=True, 
            verbose=False
            )
        
        # resample to SAMPLING_FREQUENCY and update fs
        wave = sound.resample(
            s=wave,
            fs=fs,
            target_fs=CONFIG['sampling_frequency'],
            )
        fs = CONFIG['sampling_frequency']
        duration = len(wave) / fs

        # bandpass filter between BW_FREQ_MIN and BW_FREQ_MAX
        if (DATASET['flim_min'] is not None) and (DATASET['flim_max'] is not None):   
            wave = sound.select_bandwidth(
                x=wave, 
                fs=fs, 
                fcut=(DATASET['flim_min'],DATASET['flim_max'] ), 
                forder = 5,
                ftype='bandpass')
        elif DATASET['flim_max'] is not None:
            wave = sound.select_bandwidth(
                x=wave, 
                fs=fs, 
                fcut=(DATASET['flim_max']), 
                forder = 5,
                ftype='low')
        elif DATASET['flim_min'] is not None :
            wave = sound.select_bandwidth(
                x=wave, 
                fs=fs, 
                fcut=(DATASET['flim_min']), 
                forder = 5,
                ftype='high')
        
        # trim
        if (DATASET['tlim_min'] is not None) and (DATASET['tlim_max'] is not None):
            wave = sound.trim(
                s=wave, 
                fs=fs,
                min_t=DATASET['tlim_min'],
                max_t=DATASET['tlim_max'] )
        elif (DATASET['tlim_min'] is not None):
            wave = sound.trim(
                s=wave, 
                fs=fs,
                min_t=DATASET['tlim_min'],
                max_t=duration)
        elif (DATASET['tlim_max'] is not None):
            wave = sound.trim(
                s=wave, 
                fs=fs,
                min_t=0,
                max_t=DATASET['tlim_max'] )

        """ ===================================================================
                        Computation in the time domain 
        ==================================================================="""

        # compute all the audio indices and store them into a DataFrame
        # dB_threshold and rejectDuration are used to select audio events.
        df_audio_ind = features.all_temporal_alpha_indices(
            wave, fs,
            mode=CONFIG['mode_env'],
            Nt=CONFIG['Nt'],
            gain=CONFIG['gain'],
            sensibility=CONFIG['sensibility'],
            Vadc=CONFIG['sensibility'],
            dt=CONFIG['deltaT'],
            dB_threshold=CONFIG['dB_threshold'],
            rejectDuration=CONFIG['reject_duration'],
            verbose=VERBOSE,
            display=False
            )

        # Test if the audio clip
        if ((wave.max() >= 0.99) or (wave.min() <= -0.99)):
            clip = 1
        else:
            clip = 0
            
        df_audio_ind.insert(
            loc=0, 
            column='clipping', 
            value=clip
            )
        
        df_audio_ind.insert(
            loc=1, 
            column='audio_duration', 
            value=len(wave) / fs /60
            )

        """ ==================================================================
                        Computation in the frequency domain 
        ==================================================================="""

        # Compute the Power Spectrogram Density (PSD) : Sxx_power
        Sxx_power, tn, fn, ext = sound.spectrogram(
            wave,fs,
            window=CONFIG['window'],
            nperseg=CONFIG['n_fft'],
            noverlap=CONFIG['hop_length'],
            verbose=VERBOSE,
            display=False,
            savefig=None)

        # compute all the spectral indices and store them into a DataFrame
        # flim_low, flim_mid, flim_hi corresponds to the frequency limits in Hz
        # that are required to compute somes indices (i.e. NDSI)
        # if R_compatible is set to 'soundecology', then the output is similar to
        # soundecology R package.
        df_spec_ind, _ = features.all_spectral_alpha_indices(
            Sxx_power,
            tn, fn,
            flim_low=CONFIG['flim_low'],
            flim_mid=CONFIG['flim_mid'],
            flim_hi=CONFIG['flim_hi'],
            R_compatible='soundecology',
            seed_level=CONFIG['seed_level'], 
            low_level=CONFIG['low_level'], 
            fusion_rois=CONFIG['fusion_rois'],
            remove_rois_flim_min = CONFIG['remove_rois_flim_min'],
            remove_rois_flim_max = CONFIG['remove_rois_flim_max'],
            remove_rain = CONFIG['remove_rain'],
            min_event_duration=CONFIG['min_event_duration'], 
            max_event_duration=CONFIG['max_event_duration'], 
            min_freq_bw=CONFIG['min_freq_bw'], 
            max_freq_bw=CONFIG['max_freq_bw'], 
            max_ratio_xy = CONFIG['max_ratio_xy'],
            verbose=VERBOSE,
            display=False)

        """ ===================================================================
                        Create a dataframe 
        ===================================================================="""        
        # add scalar indices into the df_indices dataframe
        df_indices = pd.concat([df_audio_ind,
                                df_spec_ind], axis=1)

        # add date and audio_path
        df_indices.insert(0, 'Date', date_time)
        df_indices.insert(1, 'file', audio_full_filename)
    
    except:
        # create a new dataframe with the column Date and file
        df_indices = pd.DataFrame({'Date': [date_time], 'fullfilename': [audio_full_filename], 'file': [file]})

    return df_indices



## 3. Process all the datasets

In [ ]:
"""==============================================================================
    Main loop to process all 1-min soundscapes
    ------------------------------------------

    Summary of the steps:
    ---------------------
    - for each dataset in the configuration file:
        - prepare the configuration for the dataset
        - parse a directory in order to get a df with date and fullfilename
        - Multi CPU processing to compute acoustic indices for each audio file
        - save the dataframe with all indices
        
=============================================================================="""

if PROCESS_DATA :

    for DATASET in CONFIG['datasets']:

        """ ===========================================================================
                        Prepare the configuration for the dataset
        ============================================================================"""
        
        # test if audio_extension exists in the dataset configuration
        if 'audio_extension' not in DATASET.keys():
            DATASET['audio_extension'] = CONFIG['audio_extension']

        # display the dataset name
        display(f'Dataset {DATASET["name"]} is being processed...')
        # parse a directory in order to get a df with date and fullfilename
        df = util.date_parser(
                        datadir=DATASET['path'], 
                        dateformat=DATASET['datetime_format'], 
                        extension=DATASET['audio_extension'],
                        verbose=False)
        # Date is used as index. Reset the index in order to get back Date as column
        df.reset_index(inplace = True)
        # number of files to process
        display('{} files will be processed'.format(len(df)))
    
        """ ===========================================================================
                        Multi CPU
        ============================================================================"""
        # At least 2 CPUs will be used in parallel and the files to process will be 
        # distributed on each CPU depending on their availability. This will speed up
        # the process.

        # create an empty dataframe. It will contain all indices for each
        # audio file in the directory
        df_indices = pd.DataFrame()

        # create an empty dataframe. It will contain files that failed
        df_failed = pd.DataFrame()

        # Number of CPU used for the calculation. 
        nb_cpu = np.max([2, os.cpu_count()])

        tic = time.perf_counter()
        # Multicpu process
        with tqdm(total=len(df), desc="multi cpu indices calculation...") as pbar:
            with futures.ProcessPoolExecutor(max_workers=nb_cpu-1) as pool:
                # give the function to map on several CPUs as well its arguments as 
                # as list
                for df_indices_temp in pool.map(
                        single_file_processing, 
                        df["file"].to_list(), 
                        df["Date"].to_list()
                        ):
                    pbar.update(1)
                    # test the number of columns. If only two, the process failed for the audio file
                    if len(df_indices_temp.columns) == 3:
                        df_failed = pd.concat([df_failed, df_indices_temp], axis=0)
                    else :
                        df_indices = pd.concat([df_indices, df_indices_temp], axis=0)
        
        df_indices.set_index('Date', inplace=True)
        
        toc = time.perf_counter()
        
        # time duration of the process
        multicpu_duration = toc - tic

        print(f"Elapsed time is {multicpu_duration:0.1f} seconds")

        # display the files that failed to be processed (df_failed)
        print(f"The number of audio files that failed to be processes is {len(df_failed)}")
        display(df_failed)

        if SAVE == True:
            # save df_indices
            df_indices.to_csv(
                            path_or_buf=os.path.join(
                                                CONFIG['save_dir'], 
                                                'indices_'+DATASET['name']+'_BW'+str(DATASET['flim_min'])+'Hz_'+str(DATASET['flim_max'])+'Hz_'+str(CONFIG['seed_level'])+'db'+'.csv'), 
                            sep=',', 
                            mode='w', 
                            header=True, 
                            index=True)